In [ ]:
import torch
torch.cuda.is_available()

In [1]:
import os
from typing import List
from tqdm import tqdm
import fire
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoModelForCausalLM,  BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from fed_utils import FedAvg, client_selection, global_evaluation, GeneralClient
import datasets
from utils.prompter import Prompter
datasets.utils.logging.set_verbosity_error()


In [ ]:
# Load the tokenizer and model
tokenizer = LlamaTokenizer.from_pretrained(
    "chavinlo/alpaca-native", cache_dir="/dpc/kunf0007/amine"
)
model = AutoModelForCausalLM.from_pretrained(
    "chavinlo/alpaca-native", cache_dir="/dpc/kunf0007/amine"
)

In [ ]:
# Encode some text
text = "What are all the positions in hockey?"
input_ids = tokenizer.encode(text, return_tensors="pt")

In [ ]:
# Generate predictions
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

In [ ]:
# Decode the generated tokens to get the text
predicted_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(predicted_text)

In [3]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [5]:
local_path = "/dpc/kunf0007/amine/lora-shepherd-7b/1/trainer_saved/local_output_0/checkpoint-400/"
tokenizer_ = LlamaTokenizer.from_pretrained(local_path, cache_dir="/dpc/kunf0007/amine")
model_ = AutoModelForCausalLM.from_pretrained(
    local_path,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    cache_dir="/dpc/kunf0007/amine/lora-shepherd-7b/",
    low_cpu_mem_usage=True
)

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

SafetensorError: Error while deserializing header: InvalidHeaderDeserialization

In [ ]:
# Encode some text
text_ = "What are all the positions in hockey?"
input_ids_ = tokenizer_.encode(text_, return_tensors="pt")

# Generate predictions
output_ = model_.generate(input_ids_, max_length=50, num_return_sequences=1)

# Decode the generated tokens to get the text
predicted_text_ = tokenizer_.decode(output_[0], skip_special_tokens=True)
print(predicted_text_)